# Wordcount

## Complete version

In [ ]:
import os

In [ ]:
import pycompss.interactive as ipycompss

In [ ]:
from pycompss.api.task import task

In [ ]:
from pycompss.api.parameter import *

In [ ]:
if 'BINDER_SERVICE_HOST' in os.environ:
    ipycompss.start(graph=True, trace=True, debug=False,
                    project_xml='../xml/project.xml',
                    resources_xml='../xml/resources.xml')
else:
    ipycompss.start(graph=True, monitor=1000, trace=True, debug=False)

In [ ]:
@task(returns=list)
def read_file(file_path):
    """ Read a file and return a list of words.
    :param file_path: file's path
    :return: list of words
    """
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data += line.split()
    return data

In [ ]:
@task(returns=dict)
def wordCount(data):
    """ Construct a frequency word dictorionary from a list of words.
    :param data: a list of words
    :return: a dictionary where key=word and value=#appearances
    """
    partialResult = {}
    for entry in data:
        if entry in partialResult:
            partialResult[entry] += 1
        else:
            partialResult[entry] = 1
    return partialResult

In [ ]:
@task(returns=dict, priority=True)
def merge_dicts(*dictionaries):
    import Queue
    q = Queue.Queue()
    for i in dictionaries:
        q.put(i)
    while not q.empty():
        x = q.get()
        if not q.empty():
            y = q.get()
            for k in y:
                if k in x:
                    x[k] += y[k]
                else:
                    x[k] = y[k]
            q.put(x)
    return(x)    

In [ ]:
from pycompss.api.api import compss_wait_on

# Get the dataset path
pathDataset = os.getcwd() + '/dataset'

# Construct a list with the file's paths from the dataset
partialResult = []
for fileName in os.listdir(pathDataset):
    p = os.path.join(pathDataset, fileName)
    data=read_file(p)
    partialResult.append(wordCount(data))
    
# Accumulate the partial results to get the final result.
result=merge_dicts(*partialResult)

# Wait for result
result = compss_wait_on(result)

In [ ]:
print("Result:")
from pprint import pprint
pprint(result)
print("Words: {}".format(sum(result.values())))

In [ ]:
ipycompss.stop()